# Notebook 04: CNN 1D Model

## Objectives
1. Load sequential features
2. Build and train CNN 1D model
3. Monitor training progress
4. Evaluate performance

In [ ]:
import sys
sys.path.append('..')

from src.utils.helpers import load_config, set_random_seeds, save_results
from src.models.cnn_1d import CNN1DSpeakerClassifier
from src.training.trainer import ModelTrainer
from src.evaluation.metrics import evaluate_model
from src.evaluation.visualization import plot_confusion_matrix, plot_per_speaker_accuracy, plot_training_history
from src.data.dataset import FeatureDataset
import matplotlib.pyplot as plt

config = load_config('../config/config.yaml')
set_random_seeds(config['seeds']['tensorflow'])

In [ ]:
# Load features
train_dataset = FeatureDataset.load_hdf5('../data/processed/train_sequential.h5')
X_train, y_train = train_dataset.get_data()

val_dataset = FeatureDataset.load_hdf5('../data/processed/val_sequential.h5')
X_val, y_val = val_dataset.get_data()

test_dataset = FeatureDataset.load_hdf5('../data/processed/test_sequential.h5')
X_test, y_test = test_dataset.get_data()

num_speakers = len(set(y_train))
input_shape = X_train.shape[1:]
print(f'Training: {X_train.shape}, Validation: {X_val.shape}, Test: {X_test.shape}')
print(f'Number of speakers: {num_speakers}')
print(f'Input shape: {input_shape}')

In [ ]:
# Create and train model
model = CNN1DSpeakerClassifier(num_speakers, input_shape, config)
model.build()

In [ ]:
# Train model
trainer = ModelTrainer(model, config)
history = trainer.train(X_train, y_train, X_val, y_val, save_path='../models/cnn_best.h5')

In [ ]:
# Plot training history
fig = plot_training_history(
    history,
    metrics=['accuracy', 'loss'],
    save_path='../results/cnn/training_history.png'
)
plt.show()

In [ ]:
# Evaluate on test set
test_results = evaluate_model(model, X_test, y_test)

print('\nTest Results:')
for metric, value in test_results['metrics'].items():
    print(f'  {metric}: {value:.4f}')

In [ ]:
# Plot confusion matrix
fig = plot_confusion_matrix(
    test_results['confusion_matrix_normalized'],
    normalize=True,
    title='CNN 1D - Confusion Matrix',
    save_path='../results/cnn/confusion_matrix.png'
)
plt.show()

In [ ]:
# Plot per-speaker accuracy
fig = plot_per_speaker_accuracy(
    test_results['per_speaker_accuracy'],
    title='CNN 1D - Per-Speaker Accuracy',
    save_path='../results/cnn/per_speaker_accuracy.png'
)
plt.show()

In [ ]:
# Save results
save_results(test_results['metrics'], '../results/cnn/test_metrics.json')
save_results(history, '../results/cnn/training_history.json')
print('\nResults saved!')